<a href="https://colab.research.google.com/github/garry-p12/PhrasalSegmentation/blob/main/DSC253_HW2_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shangjingbo1226/AutoPhrase.git

Cloning into 'AutoPhrase'...
remote: Enumerating objects: 967, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 967 (delta 66), reused 122 (delta 60), pack-reused 830 (from 1)
Receiving objects: 100% (967/967), 199.80 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (438/438), done.


In [3]:
!cd AutoPhrase/ && ./auto_phrase.sh

===Compilation===
mkdir -p bin
g++ -std=c++11 -Wall -O3 -msse2  -fopenmp  -I.. -pthread -lm -Wno-unused-result -Wno-sign-compare -Wno-unused-variable -Wno-parentheses -Wno-format -o bin/segphrase_train src/main.cpp
g++ -std=c++11 -Wall -O3 -msse2  -fopenmp  -I.. -pthread -lm -Wno-unused-result -Wno-sign-compare -Wno-unused-variable -Wno-parentheses -Wno-format -o bin/segphrase_segment src/segment.cpp
===Downloading Toy Dataset===
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198M  100  198M    0     0  3819k      0  0:00:53  0:00:53 --:--:-- 4070k
===Tokenization===

real	4m49.176s
user	7m48.369s
sys	0m17.651s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spen

In [5]:
!cd AutoPhrase/ && ./phrasal_segmentation.sh

===Compilation===
===Tokenization===

real	1m46.946s
user	2m23.844s
sys	0m15.585s
Detected Language: EN
===Part-Of-Speech Tagging===

===Phrasal Segmentation===
=== Current Settings ===
Segmentation Model Path = models/DBLP/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.500000
	Q(single-word phrases) >= 0.800000
POS guided model loaded.
# of loaded patterns = 721623
# of loaded truth patterns = 234490
POS transition matrix loaded
Phrasal segmentation finished.
   # of total highlighted quality phrases = 24355353
   # of total processed sentences = 13186794
   avg highlights per sentence = 1.84695

real	4m29.229s
user	4m19.691s
sys	0m6.549s
===Generating Output===

real	1m55.947s
user	1m49.747s
sys	0m6.818s


In [1]:
import re

def parse_segmented_corpus(input_file, output_file):
    sentences = []

    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            # Replace phrases with a single token (remove <phrase> tags and replace spaces with underscores)
            cleaned_line = re.sub(r'<phrase_Q=[^>]+>([^<]+)</phrase>', lambda m: m.group(1).replace(' ', '_'), line)
            sentences.append(cleaned_line.split())

    # Write the parsed corpus into a new file
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for sentence in sentences:
            out_f.write(" ".join(sentence) + "\n")

    return sentences

parsed_sentences = parse_segmented_corpus('/content/segmentation.txt', 'parsed_corpus.txt')

print(parsed_sentences[0])


['OQL[C++]:', 'Extending', 'C++', 'with', 'an', 'Object', 'Query', 'Capability.']


In [3]:
print(parsed_sentences[10])

['Where', 'Object-Oriented', 'DBMSs', 'Should', 'Do', 'Better:', 'A', 'Critique', 'Based', 'on', 'Early_Experiences.']


In [5]:
from gensim.models import Word2Vec

model = Word2Vec(parsed_sentences, vector_size=100, window=5, min_count=5, workers=4)

In [6]:
model.save("word2vec_phrases_parsed.model")

In [7]:
import numpy as np
model = Word2Vec.load("/content/word2vec_phrases_parsed.model")

phrases = list(model.wv.index_to_key)
embeddings = np.array([model.wv[phrase] for phrase in phrases])

In [17]:
phrases[:30]

['the',
 'of',
 'and',
 'a',
 'to',
 '.',
 'in',
 'for',
 'is',
 'that',
 'on',
 'The',
 'with',
 'are',
 'by',
 'this',
 'as',
 'an',
 'we',
 'be',
 'We',
 'A',
 'can',
 'In',
 'which',
 'from',
 'This',
 'paper',
 'using',
 'system']

In [9]:
from sklearn.cluster import KMeans

# Set number of clusters to 6
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(embeddings)

# Get cluster labels
kmeans_labels = kmeans.labels_

In [11]:
from sklearn.mixture import GaussianMixture

# Fit a Gaussian Mixture Model with 6 components (clusters)
gmm = GaussianMixture(n_components=6, random_state=42)
gmm.fit(embeddings)

# Get cluster labels
gmm_labels = gmm.predict(embeddings)


In [12]:
def print_cluster_phrases(phrases, labels, method_name, cluster_num=6, num_phrases=20):
    for i in range(cluster_num):
        print(f"\nCluster {i+1} - {method_name}:")
        cluster_phrases = [phrases[j] for j in range(len(phrases)) if labels[j] == i]
        print(cluster_phrases[:num_phrases])  # Print top 20 phrases from each cluster

# Print results for K-Means
print_cluster_phrases(phrases, kmeans_labels, 'K-Means')

print_cluster_phrases(phrases, gmm_labels, 'GMM')


Cluster 1 - K-Means:
['System', 'Design', 'Analysis', 'Approach', 'New', 'Model', 'Systems.', 'Algorithm', 'Distributed', 'Data', 'Information', 'System.', 'Dynamic', 'Learning', 'Efficient', 'Parallel', 'Application', 'Method', 'Modeling', 'Mobile']

Cluster 2 - K-Means:
['-', 'Simulation', 'recently', 'Inc.', 'VLSI', 'FPGA', 'IEEE', '&', 'et_al.', 'named', 'multiprocessor', 'First', ':', 'Association_for_Computing_Machinery.', 'Over', 'IBM', 'protocol,', 'inspired', 'conjunction', 'fault-tolerant']

Cluster 3 - K-Means:
['the', 'of', 'a', 'to', 'for', 'is', 'that', 'on', 'The', 'with', 'are', 'by', 'as', 'an', 'we', 'be', 'We', 'A', 'can', 'which']

Cluster 4 - K-Means:
['.', 'through', 'called', 'system.', 'systems.', 'applications.', 'via', 'problem.', 'presented.', 'data.', 'networks.', 'model.', 'time.', 'approach.', 'environment.', 'process.', 'problems.', 'performance.', 'algorithm.', 'network.']

Cluster 5 - K-Means:
['Using', 'An', 'On', 'Based', 'presents', 'With', 'propose